In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
import os
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image, ImageOps
import pickle

#clean images with open cv and imagehdr
import cv2
import imghdr

In [3]:
#create variable for data directory
data_direct = 'Resources/'
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [4]:
for image_class in os.listdir(data_direct):
    for image in os.listdir(os.path.join(data_direct, image_class)):
        print(image)

1716.jpeg
1346.jpeg
373.jpeg
ea36b40d2df2003ed1584d05fb1d4e9fe777ead218ac104497f5c978a7e8b7bc_640.jpg
e831b40a2cf2023ed1584d05fb1d4e9fe777ead218ac104497f5c978a7eebdbb_640.jpg
1653.jpeg
ea35b70e2bf3033ed1584d05fb1d4e9fe777ead218ac104497f5c978a7eebdbb_640.jpg
1203.jpeg
1829.jpeg
47.jpeg
1311.jpeg
ea36b30629f3013ed1584d05fb1d4e9fe777ead218ac104497f5c978a7e8b7bc_640.jpg
ea37b90c20f0033ed1584d05fb1d4e9fe777ead218ac104497f5c978a7eebdbb_640.jpg
774.jpeg
ea37b40d2cf1083ed1584d05fb1d4e9fe777ead218ac104497f5c978a7ebb0bb_640.jpg
1254.jpeg
1604.jpeg
631.jpeg
ea35b0082df7033ed1584d05fb1d4e9fe777ead218ac104497f5c978a7ebb0bb_640.jpg
1487.jpeg
148.jpeg
10.jpeg
ea37b90b2bf0013ed1584d05fb1d4e9fe777ead218ac104497f5c978a7ebb0bb_640.jpg
1192.jpeg
1038.jpeg
298.jpeg
1757.jpeg
1307.jpeg
332.jpeg
627.jpeg
1612.jpeg
freddie-marriage-40645-unsplash.jpg
1242.jpeg
1491.jpeg
1184.jpeg
918.jpeg
ea36b5082ff2093ed1584d05fb1d4e9fe777ead218ac104497f5c978a7eebdbb_640.jpg
1350.jpeg
1700.jpeg
1215.jpeg
220.jpeg
1645.jpeg


In [5]:
os.listdir(data_direct)

['cat',
 'donkey',
 'raccoon',
 'pig',
 'dog',
 'coyote',
 'fox',
 'turkey',
 'boar',
 'hare',
 'sheep',
 'duck',
 'ox',
 'owl',
 'rat',
 'goat',
 'mouse',
 'wolf',
 'chicken',
 '.ipynb_checkpoints',
 'hedgehog',
 'horse',
 'squirrel',
 'bison',
 'cow',
 'deer',
 'goose',
 'snake',
 'possum']

In [6]:
# check if image loads in open cv and remove unusable images
for image_class in os.listdir(data_direct):
    for image in os.listdir(os.path.join(data_direct, image_class)):
        image_path = os.path.join(data_direct, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print("Image extentsion not allowed {}".format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("There is an issue with image {}".format(image_path))

In [7]:
for image_class in os.listdir(data_direct):
    for image in os.listdir(os.path.join(data_direct, image_class)):
        image_path = os.path.join(data_direct, image_class, image)
        im = Image.open(image_path)
        ImageOps.pad(im, (224,224), color='black').save(image_path)

In [10]:
class_names = ['bison', 'boar', 'cat', 'chicken', 'cow', 'coyote', 'deer', 'dog', 'donkey', 'duck', 'fox', 'goat', 'goose', 
               'hare', 'hedgehog', 'horse', 'mouse', 'owl', 'ox', 'pig', 'possum', 'raccoon', 'rat', 'sheep', 'snake', 'squirrel', 
               'turkey', 'wolf']

training_images = tf.keras.utils.image_dataset_from_directory(data_direct,
                                                                      labels='inferred',
                                                                      class_names=class_names,
                                                                      batch_size = 32,
                                                                      image_size=(224, 224),
                                                                      validation_split=0.3,
                                                                      subset='training',
                                                                      seed=123,
                                                                      pad_to_aspect_ratio=True)

validation_images = tf.keras.utils.image_dataset_from_directory(data_direct,
                                                                        labels='inferred',
                                                                        class_names=class_names,
                                                                      batch_size = 32,
                                                                      image_size=(224, 224),
                                                                      validation_split=0.3,
                                                                      subset='validation',
                                                                      seed=123,
                                                                      pad_to_aspect_ratio=True)

Found 17258 files belonging to 28 classes.
Using 12081 files for training.
Found 17258 files belonging to 28 classes.
Using 5177 files for validation.


In [19]:
vgg16_model = tf.keras.applications.vgg16.VGG16()
animal_model = Sequential()
for layer in vgg16_model.layers[:-1]:
    animal_model.add(layer)
animal_model.add(Dense(units=28, activation='softmax'))
animal_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
animal_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 28)             │       114,716 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,375,260 (512.60 MB)

 Trainable params: 134,375,260 (512.60 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
AUTOTUNE = tf.data.AUTOTUNE

training_images = training_images.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_images = validation_images.cache().prefetch(buffer_size=AUTOTUNE)

In [28]:
history = animal_model.fit(training_images,
                validation_data = validation_images,
                epochs=3,
                verbose=True)

Epoch 1/3
 21/378 ━━━━━━━━━━━━━━━━━━━━ 34:43 6s/step - accuracy: 0.9309 - loss: 0.2380

KeyboardInterrupt: 

In [ ]:
# save the animal classification model as a pickle file
model_pkl_file = "animal_classifier_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(animal_model, file)

In [ ]:
# # load model from pickle file
# with open(model_pkl_file, 'rb') as file:  
#     model = pickle.load(file)
    
# test_predict = model.predict('')  